# ACS2 in Multiplexer

In [1]:
%matplotlib inline

# General
from __future__ import unicode_literals

import numpy as np
import pandas as pd

# Logger
import logging
logging.basicConfig(level=logging.WARN)

# ALCS + custom environments
import sys
sys.path.append('/Users/khozzy/Projects/pyalcs')
#sys.path.append("/Users/khozzy/Projects/openai-envs")

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Load PyALCS module
from alcs import ACS2, ACS2Configuration

# Load environments
import gym
import gym_multiplexer

## Multiplexer

In [2]:
mp = gym.make('boolean-multiplexer-20bit-v0')
situation = mp.reset()
mp.render()

'011100101010110100110'

In [3]:
state, reward, done, _ = mp.step(0)

print("New state: {}, reward: {}, is done: {}".format(state, reward, done))

New state: 011100101010110100111, reward: 1000, is done: True


In [4]:
mp.render()

'011100101010110100111'

### Evaluation
Calculate knowledge - check if any of the reliable classifiers gives correct answer.

In [5]:
from gym_multiplexer.utils import get_correct_answer

def evaluate_performance(env, population, ctrl_bits):
    p1 = env.render()  # state after executing action
    p0 = p1[:-1] + '0'  # initial state
    true_action = get_correct_answer(p0, ctrl_bits)
    
    # get all classifiers matching initial state
    matching_cls = {c for c in population if c.condition.does_match(p0)}
    best_cl = max(matching_cls, key=lambda cl: cl.q)
        
    return {'was_correct': best_cl.predicts_successfully(p0, true_action, p1)}

## Go agent, go...
Perform experiment for a couple of explore/exploit trials.

In [6]:
def get_6bit_mp_actors():
    mp = gym.make('boolean-multiplexer-6bit-v0')
    cfg = ACS2Configuration(
    mp.env.observation_space.n,
    2,
    performance_fcn=evaluate_performance,
    performance_fcn_params={'ctrl_bits': 2},
    do_ga=True)

    return ACS2(cfg), mp

def get_11bit_mp_actors():
    mp = gym.make('boolean-multiplexer-11bit-v0')
    cfg = ACS2Configuration(
    mp.env.observation_space.n,
    2,
    performance_fcn=evaluate_performance,
    performance_fcn_params={'ctrl_bits': 3},
    do_ga=True)

    return ACS2(cfg), mp

def get_20bit_mp_actors():
    mp = gym.make('boolean-multiplexer-20bit-v0')
    cfg = ACS2Configuration(
    mp.env.observation_space.n,
    2,
    performance_fcn=evaluate_performance,
    performance_fcn_params={'ctrl_bits': 4},
    do_ga=True)

    return ACS2(cfg), mp

def get_37bit_mp_actors():
    mp = gym.make('boolean-multiplexer-37bit-v0')
    cfg = ACS2Configuration(
    mp.env.observation_space.n,
    2,
    performance_fcn=evaluate_performance,
    performance_fcn_params={'ctrl_bits': 5},
    do_ga=True)

    return ACS2(cfg), mp

In [7]:
def perform_experiment(agent, env, trials=500_000):
    population, metrics = agent.explore_exploit(env, trials)
    print("Population size: {}".format(metrics[-1]['agent']['population']))
    print("Reliable size: {}".format(metrics[-1]['agent']['reliable']))
    print(metrics[-1])
    
    reliable_classifiers = [c for c in population if c.is_reliable()]
    reliable_classifiers = sorted(reliable_classifiers, key=lambda cl: -cl.q)

    # Print top 20 reliable classifiers
    for cl in reliable_classifiers[:20]:
        print(f"{cl}, q: {cl.q:.2f}, exp: {cl.exp:.2f}")
    
    return population, metrics

In [8]:
# 6bit
p6, m6 = perform_experiment(*get_6bit_mp_actors())

Population size: 72
Reliable size: 21
{'agent': {'population': 72, 'numerosity': 304, 'reliable': 21, 'fitness': 465.78839710217085, 'trial': 499999, 'steps': 1, 'total_steps': 500000}, 'environment': None, 'performance': {'was_correct': False}}
01#1###-0-####### @ 0x114332e10, q: 1.00, exp: 8519.00
10##1##-0-####### @ 0x110e97240, q: 1.00, exp: 8458.00
01#0###-0-######1 @ 0x11476ef28, q: 1.00, exp: 22771.00
11###0#-0-######1 @ 0x1148687b8, q: 1.00, exp: 22774.00
000####-0-######1 @ 0x114868908, q: 1.00, exp: 22804.00
10##0##-0-######1 @ 0x114868400, q: 1.00, exp: 22574.00
11###1#-0-####### @ 0x11475ccc0, q: 1.00, exp: 8391.00
001####-1-######1 @ 0x11476e2e8, q: 1.00, exp: 22687.00
001####-0-####### @ 0x11475c908, q: 1.00, exp: 8506.00
10##1##-1-######1 @ 0x114868fd0, q: 1.00, exp: 22515.00
11###1#-1-######1 @ 0x110e97c18, q: 1.00, exp: 22404.00
10##0##-1-####### @ 0x114332358, q: 1.00, exp: 8535.00
11###0#-1-####### @ 0x11475ce48, q: 1.00, exp: 8500.00
000####-1-####### @ 0x11476ec50,

In [9]:
# 11bit
p11, m11 = perform_experiment(*get_11bit_mp_actors())

Population size: 170
Reliable size: 43
{'agent': {'population': 170, 'numerosity': 594, 'reliable': 43, 'fitness': 483.8730054665155, 'trial': 499999, 'steps': 1, 'total_steps': 500000}, 'environment': None, 'performance': {'was_correct': False}}
101#####0###-1-############ @ 0x13236a2e8, q: 1.00, exp: 4136.00
101#####0###-0-###########1 @ 0x1326466d8, q: 1.00, exp: 11472.00
0001########-0-############ @ 0x1325540f0, q: 1.00, exp: 4205.00
111#######1#-0-############ @ 0x132798668, q: 1.00, exp: 4140.00
010##1######-0-############ @ 0x132505b70, q: 1.00, exp: 4204.00
001#0#######-1-############ @ 0x132315940, q: 1.00, exp: 4038.00
011###0#####-0-###########1 @ 0x132885198, q: 1.00, exp: 11377.00
010##1######-1-###########1 @ 0x11476efd0, q: 1.00, exp: 11168.00
100####0####-1-############ @ 0x1326a5ba8, q: 1.00, exp: 3995.00
0000########-1-############ @ 0x1323b9a90, q: 1.00, exp: 4050.00
111#######0#-0-###########1 @ 0x1325f8f60, q: 1.00, exp: 11317.00
100####1####-1-###########1 @ 0x13

In [10]:
# 20bit
p20, m20 = perform_experiment(*get_20bit_mp_actors())

Population size: 5888
Reliable size: 66
{'agent': {'population': 5888, 'numerosity': 6719, 'reliable': 66, 'fitness': 351.2487002352743, 'trial': 499999, 'steps': 1, 'total_steps': 500000}, 'environment': None, 'performance': {'was_correct': False}}
00001################-1-####################1 @ 0x11476e828, q: 1.00, exp: 5480.00
0110######1##########-1-####################1 @ 0x153cd0be0, q: 1.00, exp: 5179.00
1110##############1##-0-##################### @ 0x153a94ba8, q: 1.00, exp: 1979.00
1010##########1######-0-##################### @ 0x152be5780, q: 1.00, exp: 1963.00
0001#0###############-1-##################### @ 0x152d2a358, q: 1.00, exp: 1784.00
1111###############0#-1-##################### @ 0x1520bfda0, q: 1.00, exp: 1908.00
1011###########0#####-0-####################1 @ 0x155eafef0, q: 1.00, exp: 5204.00
0001#1###############-1-####################1 @ 0x15152f470, q: 1.00, exp: 4989.00
1100############0####-0-####################1 @ 0x152883b70, q: 1.00, exp: 4845.00
001

In [ ]:
# 37bit
p37, m37 = perform_experiment(*get_37bit_mp_actors())

In [ ]:
def parse_metrics_to_df(metrics):
    def extract_details(row):
        row['trial'] = row['agent']['trial']
        row['numerosity'] = row['agent']['numerosity']
        row['reliable'] = row['agent']['reliable']
        row['was_correct'] = row['performance']['was_correct']
        return row
    
    df = pd.DataFrame(metrics)
    df = df.apply(extract_details, axis=1)
    df.drop(['agent', 'environment', 'performance'], axis=1, inplace=True)
    df.set_index('trial', inplace=True)
    
    return df

In [ ]:
# parse metrics to df
df6bit = parse_metrics_to_df(m6)
df11bit = parse_metrics_to_df(m11)
df20bit = parse_metrics_to_df(m20)
df37bit = parse_metrics_to_df(m37)

df6bit.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr

window=5000

def func(x, pos):
    return x / 100_000

fig, ax = plt.subplots()

df6bit['reliable'].head(300_000).rolling(window=window).mean().plot(color='r', linewidth=1.0, ax=ax)
ax.text(275_000, 29, '6-bit', color='r')

df11bit['reliable'].head(300_000).rolling(window=window).mean().plot(color='b', linewidth=1.0, ax=ax)
ax.text(275_000, 51, '11-bit', color='b')

df20bit['reliable'].head(300_000).rolling(window=window).mean().plot(color='g', linewidth=1.0, ax=ax)
ax.text(275_000, 85, '20-bit', color='g')

ax.set_xlabel('Trial (x 100 000)')
ax.set_ylabel('Reliable classifiers')
ax.xaxis.set_major_formatter(tkr.FuncFormatter(func))
ax.set_title(f'Number of reliable classifiers for boolean MPX.\nResults averaged over {window} trials')
#plt.show()

plt.savefig('mpx.eps', format='eps', dpi=100)